In [ ]:
samples = []
# /path/to/fastq files
filepath = '/home/gridsan/chong/hansen_lab_shared/clarice/megapool_050624/fastq/'

# file containing R1 (read1) fastq filenames, one file per line
with open('R1_fastqs.txt', 'r') as f:
    for line in f:
        r1_filename = line.strip('\n')
        r2_filename = r1_filename.replace('R1_001', 'R2_001')
        metadata = r1_filename.split('_')
        sample = metadata[3]
        rep = metadata[4]
        lane = metadata[6]
        id = '_'.join([sample, rep, lane])
        samples.append([id, sample, rep, lane, filepath + r1_filename, filepath + r2_filename])

In [ ]:
# sample map used for Snakemake 
with open('sample_map.csv', 'w') as f:
    f.write('sample_id,sample,rep,lane,fastq_r1,fastq_r2\n')
    for i in samples:
        f.write(','.join(i) + '\n')

### Parse pairtools stats files

In [ ]:
def parse_stats(filename):
    """
    Parse output of pairtools stats function
    """
    out = {}
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip('\n').split('\t')
            if line[0] == 'total_mapped':
                out['mapped'] = int(line[1])
            elif line[0] == 'total_dups':
                out['dups'] = int(line[1])
            elif line[0] == 'total_nodups':
                out['nodups'] = int(line[1])
                break
            else:
                pass

    return out

In [ ]:
# write pairtools stats for each location and replicate to file
parsed_stats = open('pairtools_dup_stats.txt', 'w')
parsed_stats.write('sample\tmapped\tnodups\tdups\tperc_nodup\n')

with open('stats_filenames.txt', 'r') as f:
    for line in f:
        sample = line.strip('_stats.txt\n').split('/')[1]
        stats = parse_stats(line.strip('\n'))
        percentage_nodup = stats['nodups']/stats['mapped']
        parsed_stats.write('{sample}\t{mapped:,}\t{nodups:,}\t{dups:,}\t{perc:.2f}\n'.format(
            sample = sample, mapped = stats['mapped'], nodups = stats['nodups'], dups = stats['dups'],
            perc = percentage_nodup
        ))

parsed_stats.close()